In [1]:
library(devtools)
library(metacell)
load_all("../metacell.flow/")
scdb_init("output/scrna_db/",force_reinit = T)
scfigs_init("figs")
scdb_flow_init()

Loading required package: usethis

i Loading metacell.flow

initializing scdb to output/scrna_db/



In [2]:
tgconfig::set_param(param = "mcell_mc2d_max_confu_deg",value = 4, package = "metacell")

In [3]:
library("reticulate")
library("anndata")
sc = import("scanpy")
mc2 = import("metacells")
# metacells <- anndata::read_h5ad('oc_emb_roned.h5ad')
metacells <- anndata::read_h5ad('oc_exe.h5ad')
# metacells <- anndata::read_h5ad('../rabemb_2203/metacells11_2203_woexec_for_mcview_ron.h5ad')

In [4]:
mc_id = "oc_emb"

In [5]:
mat_id = "oc_emb"
mc_id = "oc_emb"

feat_gset = "oc_emb"
mgraph_id = "oc_emb"

In [6]:
mat_id = "oc_embexe"
mc_id = "oc_embexe"
mgraph_id = "oc_exe"

In [7]:
logist_loc = 1
logist_scale = 0.2
logist_eps = 4e-5
max_d_fold = 3

mc = scdb_mc(mc_id)
gset = scdb_gset(feat_gset)
feat_genes = names(gset@gene_set)
# mgraph = mc2d_comp_mgraph_param(mc, feat_genes, logist_loc, logist_scale, logist_eps, max_d_fold)

In [7]:
genes = feat_genes
loc = logist_loc
scale = logist_scale
eps = logist_eps
max_deg = 4
legc = log2(mc@e_gc[genes,] + eps)

logist_d = function(x) {
    d = abs(legc - x)
    d = plogis(d, loc, scale)
    return(colSums(d))
}
a = apply(legc, 2, logist_d)

In [8]:
oc_mat = scdb_mat(mat_id)
oc_mc = mc
oc_md = oc_mat@cell_metadata
oc_md$cellid = rownames(oc_md)

In [10]:
logist_loc = 1
logist_scale = 0.2
logist_eps = 4e-5
max_d_fold = 3

mc = scdb_mc(mc_id)
gset = scdb_gset(feat_gset)
feat_genes = names(gset@gene_set)
mgraph = mc2d_comp_mgraph_param(mc, feat_genes, logist_loc, logist_scale, logist_eps, max_d_fold)

scdb_add_mgraph(id = mgraph_id, mgraph = tgMCManifGraph(mc_id = mc_id, mgraph = mgraph))

In [16]:
mgraph = scdb_mgraph(mgraph_id)